In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import itertools
import warnings
warnings.filterwarnings('ignore')

In [4]:
# For Laptop

filename2 = '/home/jovyan/work/data/revolut/rev-notifications.csv'
rev_notifications = pd.read_csv(filename2)
df_rev_notifications = rev_notifications.copy()


# DATA PREPROCESSING

In [5]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

In [6]:
def unqiue_values_table(df):
    unq_val = df.nunique()
    unq_val_percent = 100 * df.nunique() / len(df)
    unq_val_table = pd.concat([unq_val, unq_val_percent], axis=1)
    unq_val_table_ren_columns = unq_val_table.rename(
    columns = {0 : 'Unique Values', 1 : '% of Total Values'})
    unq_val_table_ren_columns = unq_val_table_ren_columns[
        unq_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    'Unique Values', ascending=False).round(2)
    print ("Your selected dataframe has " + str(df.shape[0]) + " rows "       
        "and " + str(df.shape[1]) + " columns.")
    return unq_val_table_ren_columns

In [29]:
def date_conversion(df):
    df['created_date'] = pd.to_datetime(df['created_date'])
    return df

### NOTIFICATIONS 

In [7]:
missing_values_table(df_rev_notifications)

Your selected dataframe has 5 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [8]:
unqiue_values_table(df_rev_notifications)

Your selected dataframe has 121813 rows and 5 columns.


,Unique Values,% of Total Values
created_date,121810,100.00
user_id,18953,15.56
reason,17,0.01
channel,3,0.00
status,2,0.00


In [30]:
date_conversion(df_rev_notifications)

,reason,channel,status,user_id,created_date
0,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_7086,2018-12-02
1,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_6598,2018-12-01
2,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_4151,2018-12-04
3,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_1408,2018-12-11
4,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_6292,2018-12-14
...,...,...,...,...,...
121808,LOST_CARD_ORDER,EMAIL,SENT,user_12687,2019-05-07
121809,LOST_CARD_ORDER,EMAIL,SENT,user_16875,2019-05-10
121810,LOST_CARD_ORDER,EMAIL,SENT,user_17426,2019-05-09
121811,LOST_CARD_ORDER,EMAIL,SENT,user_15880,2019-05-10


In [31]:
df_rev_notifications.dtypes

reason                  object
channel                 object
status                  object
user_id                 object
created_date    datetime64[ns]
dtype: object

In [41]:
df = df_rev_notifications[['user_id','created_date']]
df = df.sort_values(["user_id", "created_date"], ascending = (True, True))
df['diff_in_date'] = df.groupby('user_id')['created_date'].diff().dt.days
df

,user_id,created_date,diff_in_date
7785,user_0,2018-10-31,NaN
50550,user_0,2018-11-25,25.0
28429,user_0,2018-12-11,16.0
46418,user_0,2019-01-12,32.0
30270,user_0,2019-02-01,20.0
...,...,...,...
29814,user_9999,2019-01-23,65.0
619,user_9999,2019-02-07,15.0
42376,user_9999,2019-02-13,6.0
44550,user_9999,2019-02-13,0.0


In [44]:
df.set_index('user_id', inplace = True)

In [53]:
df.loc[['user_0','user_9999'],['created_date','diff_in_date']].sort_values(["user_id","created_date"], ascending = (True))

,created_date,diff_in_date
user_id,,
user_0,2018-10-31,NaN
user_0,2018-11-25,25.0
user_0,2018-12-11,16.0
user_0,2019-01-12,32.0
user_0,2019-02-01,20.0
user_0,2019-02-01,0.0
user_0,2019-02-13,12.0
user_0,2019-02-26,13.0
user_9999,2018-10-19,NaN
